# TASK 1

Import libraries for all three tasks.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle json files

import re #library for regular expression operations to search strings

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Scrape the wikipedia page.

In [3]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
    'User-Agent': 'Mozilla/5.0'
}
page = requests.post(wiki, headers=header)
soup = BeautifulSoup(page.content)

Focus on the content of tables.

In [6]:
tables = soup.find("table")
tables = tables.findAll('p')

Write the information of the wikipedia table to the dataframe.

In [33]:
columns = ["PostalCode","Borough","Neighborhood"]
df = pd.DataFrame(columns = columns)
df

find_borough = re.compile(r'[>](.*?)[<]', re.S)
find_neighborhood = re.compile(r'[(](.*?)[)]', re.S)

x = 0
b = [[],[]] #Store PostalCode, Borough, and Neighborhood Data
for i,rows in enumerate(tables):
    
    #Get PostalCode and Borough
    a = re.findall(p1,str(tables[i]))
    k=0
    for j in range(len(a)):
        if a[j] != '':
            b[k].append(a[j])
            k=k+1
        if k == 2: break
    if a[j] == 'Not assigned': continue
    PostalCode, Borough = b[0][i], b[1][i]
    
    #Get Neighbourhood (between the paranthesis)
    try:
        Neighborhood = re.findall(p2,tables[i].text)[0].replace(' /',', ')
    except:
        #print(i,rows.text)
        Neighborhood = "Queen's Park, Ontario Provincial Government"
        Borough = "Downtown Toronto"
        #No neighbourhood assigned within paranthesis
        #By running the code, the problem is found only at M7A (Queen's Park / Ontario Provincial Government).
        #The problem is actually no borough is assigned. 
        #According to the wikipedia page of Queen's Park, it should be at Downtown Toronto
        #This entry is manually modified here
    
    df.loc[x]=[PostalCode, Borough, Neighborhood]
    x=x+1
   
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [37]:
df.shape

(103, 3)

# TASK 2

Get the geospatial data from the csv file.

In [12]:
df_geospatial = pd.read_csv(r'http://cocl.us/Geospatial_data')
df_geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join the geospatial data to the task 1 dataframe df.

In [117]:
df2 = df.join(df_geospatial.set_index('Postal Code'), on='PostalCode')
df2.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# TASK 3

Get the location of the Toronto City and view the neighborhoods on the map of the city.

In [36]:
address = 'Toronto City, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [17]:
# create map of Toronto City using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Input the foursquare information.

In [18]:
CLIENT_ID = '4ORWWNFRNAWZKEIWSUVLBVB4FJDXYDWYGEEYHX1PAQ1LY5NE' # your Foursquare ID
CLIENT_SECRET = 'V11YM5X3XVJGJ3R30QK0DFWC4EYZ2CFBAKHW05QQGO2ERH24' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4ORWWNFRNAWZKEIWSUVLBVB4FJDXYDWYGEEYHX1PAQ1LY5NE
CLIENT_SECRET:V11YM5X3XVJGJ3R30QK0DFWC4EYZ2CFBAKHW05QQGO2ERH24


Define the function to get nearby venues of each neighborhood. The radius is set to 1500 which is different from the lab session, to assign each neighborhood with at least one venue.

In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Scrape data from Foursquare

In [144]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park,  Harbourfront
Lawrence Manor,  Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern,  Rouge
Don Mills
Parkview Hill,  Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park,  Princess Gardens,  Martin Grove,  Islington,  Cloverdale
Rouge Hill,  Port Union,  Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate,  Bloordale Gardens,  Old Burnhamthorpe,  Markland Wood
Guildwood,  Morningside,  West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,  Wilson Heights,  Downsview North
Thorncliffe Park
Richmond,  Adelaide,  King
Dufferin,  Dovercourt Village
Scarborough Village
Fairview,  Henry Farm,  Oriole
Northwood Park,  York University
The Danforth  East
Harbourfront East,  Union Station,  Toronto Islands
Little Portugal,  Trinity
Kennedy Park,  Ionview,  East Birchmount Park
Bayview Villa

View the scraped data.

In [145]:
print(toronto_venues.shape)
toronto_venues.head()

(6924, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
4,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store


Summarize the number of venues at each neighborhood.

In [146]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,64,64,64,64,64,64
"Alderwood, Long Branch",48,48,48,48,48,48
"Bathurst Manor, Wilson Heights, Downsview North",41,41,41,41,41,41
Bayview Village,15,15,15,15,15,15
"Bedford Park, Lawrence Manor East",73,73,73,73,73,73
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",12,12,12,12,12,12
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",70,70,70,70,70,70


One hot encoding of the data.

In [147]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Gar

Summarize the one hot encoding data, the result of which can be used for clustering later.

In [148]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Gar

View the top 5 venues at each neighborhood.

In [149]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.17
1           Coffee Shop  0.06
2  Gym / Fitness Center  0.03
3  Cantonese Restaurant  0.03
4        Breakfast Spot  0.03


----Alderwood,  Long Branch----
            venue  freq
0            Bank  0.06
1     Pizza Place  0.06
2        Pharmacy  0.04
3  Discount Store  0.04
4            Café  0.04


----Bathurst Manor,  Wilson Heights,  Downsview North----
         venue  freq
0         Park  0.15
1  Coffee Shop  0.10
2  Pizza Place  0.07
3  Gas Station  0.05
4         Bank  0.05


----Bayview Village----
                 venue  freq
0                 Park  0.13
1          Gas Station  0.13
2                Trail  0.13
3  Japanese Restaurant  0.13
4                 Bank  0.13


----Bedford Park,  Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.08
2    Sushi Restaurant  0.07
3              Bakery  0.07
4          Bagel Shop  0.05


----Berczy Park----
   

                  venue  freq
0           Coffee Shop  0.11
1    Chinese Restaurant  0.08
2  Fast Food Restaurant  0.08
3        Sandwich Place  0.08
4              Pharmacy  0.08


----Kensington Market,  Chinatown,  Grange Park----
                venue  freq
0                Café  0.11
1         Coffee Shop  0.07
2                 Bar  0.06
3         Pizza Place  0.04
4  Mexican Restaurant  0.04


----Kingsview Village,  St. Phillips,  Martin Grove Gardens,  Richview Gardens----
            venue  freq
0     Coffee Shop  0.11
1        Pharmacy  0.07
2     Pizza Place  0.07
3            Bank  0.07
4  Sandwich Place  0.07


----Lawrence Manor,  Lawrence Heights----
                    venue  freq
0          Clothing Store  0.12
1    Fast Food Restaurant  0.05
2             Coffee Shop  0.05
3              Restaurant  0.05
4  Furniture / Home Store  0.03


----Lawrence Park----
                venue  freq
0         Coffee Shop  0.08
1    Sushi Restaurant  0.06
2              Bakery  0.

                  venue  freq
0                  Farm   0.5
1         National Park   0.5
2           Zoo Exhibit   0.0
3          Nudist Beach   0.0
4  Pakistani Restaurant   0.0


----Victoria Village----
                       venue  freq
0                Coffee Shop  0.15
1  Middle Eastern Restaurant  0.09
2                        Gym  0.09
3              Grocery Store  0.06
4       Fast Food Restaurant  0.06


----West Deane Park,  Princess Gardens,  Martin Grove,  Islington,  Cloverdale----
            venue  freq
0            Park  0.14
1           Hotel  0.10
2    Intersection  0.05
3        Pharmacy  0.05
4  Farmers Market  0.05


----Westmount----
           venue  freq
0           Bank  0.11
1    Pizza Place  0.09
2    Coffee Shop  0.09
3           Park  0.06
4  Grocery Store  0.06


----Weston----
           venue  freq
0  Grocery Store  0.06
1         Bakery  0.04
2  Train Station  0.04
3    Pizza Place  0.04
4           Bank  0.04


----Wexford,  Maryvale----
            

Define the function and summarize the most common venues at each neighborhood. The result is integrated into a dataframe.

In [150]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [151]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Breakfast Spot,Gym / Fitness Center,Shopping Mall,Bakery,Cantonese Restaurant,Hong Kong Restaurant,Sri Lankan Restaurant
1,"Alderwood, Long Branch",Pizza Place,Bank,Clothing Store,Burger Joint,Discount Store,Café,Toy / Game Store,Grocery Store,Pharmacy,Park
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Coffee Shop,Pizza Place,Gas Station,Bank,Ice Cream Shop,Middle Eastern Restaurant,Sushi Restaurant,Supermarket,Fried Chicken Joint
3,Bayview Village,Park,Trail,Japanese Restaurant,Bank,Gas Station,Skating Rink,Grocery Store,Chinese Restaurant,Café,Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Bakery,Sushi Restaurant,Bagel Shop,Café,Fast Food Restaurant,Restaurant,Pizza Place,Bank
5,Berczy Park,Coffee Shop,Café,Japanese Restaurant,Restaurant,Hotel,Farmers Market,Beer Bar,Park,Gastropub,Bakery
6,"Birch Cliff, Cliffside West",Park,Ice Cream Shop,Restaurant,Thai Restaurant,Gym,Gym Pool,General Entertainment,Café,Diner,Golf Course
7,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Bakery,Gift Shop,Arts & Crafts Store,Furniture / Home Store,Supermarket,Theme Park
8,"CN Tower, King and Spadina, Railway Lands, ...",Park,Coffee Shop,Café,Restaurant,Gym,Harbor / Marina,Brewery,Boat or Ferry,Track,Bar
9,Caledonia-Fairbanks,Italian Restaurant,Pizza Place,Coffee Shop,Bakery,Furniture / Home Store,Caribbean Restaurant,Pharmacy,Park,Café,Bank


Clustering of the neighborhoods.

In [152]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 2, 2, 1, 1, 2, 0, 2, 1])

Add clustering labels into the dataframe.

In [153]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Pharmacy,Coffee Shop,Supermarket,Bank,Bus Stop,Gas Station,Liquor Store,Golf Course,Sandwich Place,Caribbean Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Middle Eastern Restaurant,Gym,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Clothing Store,Supermarket,Cosmetics Shop,Thrift / Vintage Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Café,Park,Bakery,Restaurant,Thai Restaurant,Pub,Bar,Theater,Farmers Market
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Dessert Shop,Furniture / Home Store,Sandwich Place,Vietnamese Restaurant,Toy / Game Store,Turkish Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Japanese Restaurant,Restaurant,Park,Café,Gastropub,Pizza Place,Thai Restaurant,Ice Cream Shop,Bookstore
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,2,Bank,Skating Rink,Supermarket,Bakery,Liquor Store,Grocery Store,Café,Shopping Mall,Restaurant,Bus Stop
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Restaurant,Liquor Store,Paper / Office Supplies Store,Greek Restaurant,Other Great Outdoors,Bank,Caribbean Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Coffee Shop,Restaurant,Japanese Restaurant,Pizza Place,Bank,Middle Eastern Restaurant,Sandwich Place,Gym,Italian Restaurant,Park
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2,Pizza Place,Park,Fast Food Restaurant,Pharmacy,Skating Rink,Bus Line,Gastropub,Martial Arts Dojo,Breakfast Spot,Brewery
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Japanese Restaurant,Café,Restaurant,Gastropub,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Theater,Creperie


Generate the map visualizing how the neighborhoods cluster together.

In [155]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters